## Ablation - modify (a,b), (c,d) to (a,d) and (c,b) 

In [1]:
import pandas as pd
import numpy as np

In [2]:
import random

In [5]:
data_path = "../../datasets/fb15k-237/"

In [6]:
relations_df = pd.read_csv(f"{data_path}/derived/relations_with_ids.csv")

In [7]:
freebase_vs_wiki_df =  pd.read_csv(f"{data_path}/fb_wiki_mapping.tsv", sep='\t')

In [8]:
triplets_with_src_dst_wiki_ids_df = pd.read_csv(f"{data_path}/derived/triplets_with_src_dst_wiki_ids.csv")

In [9]:
# add test split with empty labels
df_test = pd.read_csv(f"{data_path}/derived/triplets_with_src_dst_wiki_ids.test.csv")

In [27]:
df = triplets_with_src_dst_wiki_ids_df
# df = df_test

In [28]:
n = len(df)
num_errors = int(0.25*n)

In [29]:
df.info()
df.timestamp.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243636 entries, 0 to 243635
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   src_freebase_id   243636 non-null  object
 1   relation          243636 non-null  object
 2   dest_freebase_id  243636 non-null  object
 3   triple_id         243636 non-null  int64 
 4   timestamp         243636 non-null  object
 5   relation_name     243636 non-null  object
 6   relation_id       243636 non-null  int64 
 7   freebase_id_x     243636 non-null  object
 8   label_x           243636 non-null  object
 9   src_wikidata_id   243636 non-null  int64 
 10  freebase_id_y     243636 non-null  object
 11  label_y           243636 non-null  object
 12  dest_wikidata_id  243636 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 24.2+ MB


timestamp
2024-11-17    2885
2024-10-18    2833
2024-12-12    2832
2024-11-14    2832
2024-12-13    2830
              ... 
2024-11-11    2654
2024-11-20    2650
2024-12-21    2648
2024-10-09    2625
2024-11-25    2613
Name: count, Length: 89, dtype: int64

In [30]:
triple_ids = list(df.triple_id.values)

In [31]:
# permute triple_ids and select top 2*num_errors
# rather select small 
selected_edges = random.sample(triple_ids, 2*num_errors)

In [32]:
perturbed_rows_dicts = []
for i in range(num_errors):
    edge_ab = int(selected_edges[2*i])
    edge_cd = int(selected_edges[2*i+1])
    if i < 5:
        print(f"swapping {edge_ab} and {edge_cd}")

    row_ab = df.loc[df['triple_id']==edge_ab]
    row_cd = df.loc[df['triple_id']==edge_cd]

    row_ab_dict = row_ab.to_dict(orient='records')[0]
    row_cd_dict = row_cd.to_dict(orient='records')[0]

    a = row_ab_dict['src_wikidata_id']
    b = row_ab_dict['dest_wikidata_id']

    c = row_cd_dict['src_wikidata_id']
    d = row_cd_dict['dest_wikidata_id']

    if i < 5:
        print(f"{a}->{b} , {c}->{d}")

    row_ad_new = row_ab_dict.copy()
    row_ad_new['dest_wikidata_id'] = d

    row_cb_new = row_cd_dict.copy()
    row_cb_new['dest_wikidata_id'] = b

    perturbed_rows_dicts.append(row_ad_new)
    perturbed_rows_dicts.append(row_cb_new)
    

swapping 67332 and 242789
219->221 , 917138->607280
swapping 8144 and 99945
43->8558 , 1204213->3180462
swapping 267908 and 20188
913440->460277 , 918484->639669
swapping 91305 and 171039
103360->152456 , 295233->1445521
swapping 138315 and 60134
2140778->4917 , 31->8088


In [33]:
# Swap endpoints for edges of same type? 


In [34]:
df_errors = pd.DataFrame.from_dict(perturbed_rows_dicts)

In [35]:
len(df_errors)

121818

In [36]:
df_clean = df[~df['triple_id'].isin(selected_edges)]

In [37]:
len(df_clean)

120940

In [38]:
# For train dataset
df_errors['disapprove'] = 1
df_clean['disapprove'] = 0

df_revised = pd.concat([df_clean, df_errors])

/var/folders/9f/_45_cttj01b5lmh5x5njvlp00000gn/T/ipykernel_99317/3297765265.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['disapprove'] = 0


In [39]:
df_revised.to_csv(f"{data_path}/exp3/triplets_with_src_dst_wiki_ids.csv", index=False)

In [25]:
# For test dataset
df_errors['disapprove'] = 1
df_clean['disapprove'] = 0

df_revised = pd.concat([df_clean, df_errors])

/var/folders/9f/_45_cttj01b5lmh5x5njvlp00000gn/T/ipykernel_99317/327292017.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['disapprove'] = 0


In [26]:
df_revised.to_csv(f"{data_path}/exp3/triplets_with_src_dst_wiki_ids.test.csv", index=False)

In [ ]:
# Evaluation on test set, do the same perturbation
# for them label to be empty per guidelines - https://kumo.ai/docs/examples/fraud-chargeback-abuse/

In [ ]:
# permute as earlier
